In [1]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
import pandas as pd

sys.path.append(os.path.abspath("D:\\burtm\\Visual_studio_code\\PD_related_projects"))

KeyboardInterrupt: 

In [3]:
selected_model = "MLP"
selected_transform = "trocr-small-stage1"
N_max=282
use_patches=True
pretrained=True
depth=2
hugging=True
num_epochs=100
batch_size=32
learning_rate=0.001
input_filename="icdar_train_df_patches_cc.csv"
criterion_name="CrossEntropyLoss"
criterion = training_utils.get_criterion(criterion_name)
optimizer_name = "Adam"
num_classes = 2  # Change this to match your dataset
early_stopping=10
scheduler_name = 'no_scheduling'#CosineAnnealingLR'
checkpoint_path = "D:\\burtm\Visual_studio_code\PD_related_projects\checkpoints\\"
models_path = "D:\\burtm\Visual_studio_code\PD_related_projects\outputs\models\\"

In [4]:
# Example training metadata
training_metadata = {
    "type_of_approach": "training mlp on top of trocr extracted features (from block 10 and last block)",
    "type_of_approach_sigla": "mlp_trocr",
    "model_name": selected_model,
    "transform_name": selected_transform,
    "epochs": num_epochs,
    "batch_size": batch_size,
    "learning_rate": learning_rate,
    "optimizer": optimizer_name,
    "pretrained": pretrained,
    "depth": depth,
    "use_patches": use_patches,
    "input_filename": input_filename,
    "num_classes": num_classes,
    "criterion_name": criterion_name,
    "early_stopping": early_stopping,
    "N_max": N_max,
    "scheduler_name": scheduler_name,
}

In [19]:

data_path="D:\\burtm\\Visual_studio_code\\PD_related_projects\\outputs\\preprocessed_data\\"
file_path_1='icdar_EXTRACTED_trocr_stage_1.csv'
file_path_2='icdar_EXTRACTED_trocr_stage_1_10blocks_20250509_153553.csv'
train_df_1 = pd.read_csv(data_path+file_path_1)
train_df_2 = pd.read_csv(data_path+file_path_2)


In [6]:
train_df_1.head()

,writer,same_text,isEng,train,file_name,male,x,y,x2,y2,...,f375,f376,f377,f378,f379,f380,f381,f382,f383,f384
0,190,0,0,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,988,494,1482,988,...,0.621636,0.495279,-0.389701,-0.058870,0.580941,-0.161189,0.491237,-0.745575,0.791981,-0.915822
1,24,0,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,990,495,1485,990,...,0.654779,0.962829,-0.261563,-0.149086,1.290398,-0.493723,0.332634,-0.845838,0.701031,-0.516267
2,26,1,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,494,1976,988,2470,...,0.538509,0.735932,-0.625200,-0.053809,1.258883,-0.857080,0.097662,-0.151117,0.769877,-1.263413
3,190,0,0,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,494,494,988,988,...,0.491024,0.589948,-0.307026,0.154349,0.862040,-0.343460,0.478519,-0.866911,0.734231,-0.810595
4,160,0,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1482,494,1976,988,...,0.903548,0.373364,-0.691305,0.246522,0.851829,0.060120,0.342756,-0.464967,0.335915,-0.447760


In [7]:
train_df_2.head()

,writer,same_text,isEng,train,file_name,male,x,y,x2,y2,...,f375,f376,f377,f378,f379,f380,f381,f382,f383,f384
0,190,0,0,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,988,494,1482,988,...,-0.922357,2.522360,4.304227,-3.425128,3.495620,-4.153519,-0.852054,-1.164748,-2.234782,-2.404248
1,24,0,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,990,495,1485,990,...,-0.485612,1.888572,4.129086,-3.297427,4.332079,-4.687369,-2.287635,-1.905988,-4.285491,-1.362528
2,26,1,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,494,1976,988,2470,...,-1.275970,1.497854,3.086382,-2.358793,4.704427,-5.747137,-3.386876,1.461058,-3.862585,-1.632300
3,190,0,0,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,494,494,988,988,...,-1.605459,2.483239,4.245601,-3.242137,3.766826,-4.104949,-1.693988,-1.805966,-2.415438,-1.441694
4,160,0,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1482,494,1976,988,...,-0.307438,1.057468,2.697207,-1.516698,3.344037,-4.181660,-2.132452,-1.470941,-4.696310,-0.665148


In [20]:
# Filter columns in train_df_2 that match the pattern "f(some_number)"
columns_to_append = [col for col in train_df_2.columns if col.startswith('f') and col[1:].isdigit()]

# Append these columns to train_df_1 with modified column names to avoid conflicts
train_df_2_modified = train_df_2[columns_to_append].rename(columns=lambda x: f"{x}_from_df2")
train_df_1 = pd.concat([train_df_1, train_df_2_modified], axis=1)

In [21]:
train_df_1.head()

,writer,same_text,isEng,train,file_name,male,x,y,x2,y2,...,f375_from_df2,f376_from_df2,f377_from_df2,f378_from_df2,f379_from_df2,f380_from_df2,f381_from_df2,f382_from_df2,f383_from_df2,f384_from_df2
0,190,0,0,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,988,494,1482,988,...,-0.922357,2.522360,4.304227,-3.425128,3.495620,-4.153519,-0.852054,-1.164748,-2.234782,-2.404248
1,24,0,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,990,495,1485,990,...,-0.485612,1.888572,4.129086,-3.297427,4.332079,-4.687369,-2.287635,-1.905988,-4.285491,-1.362528
2,26,1,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,494,1976,988,2470,...,-1.275970,1.497854,3.086382,-2.358793,4.704427,-5.747137,-3.386876,1.461058,-3.862585,-1.632300
3,190,0,0,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,494,494,988,988,...,-1.605459,2.483239,4.245601,-3.242137,3.766826,-4.104949,-1.693988,-1.805966,-2.415438,-1.441694
4,160,0,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1482,494,1976,988,...,-0.307438,1.057468,2.697207,-1.516698,3.344037,-4.181660,-2.132452,-1.470941,-4.696310,-0.665148


In [10]:
print(train_df_1.columns.tolist())

['writer', 'same_text', 'isEng', 'train', 'file_name', 'male', 'x', 'y', 'x2', 'y2', 'n_cc', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127',

In [24]:
transform=u_transforms.get_transform(selected_transform,use_patches=use_patches)
train_dataloader,val_dataloader=data_loading.get_dataloaders(transform, batch_size=16, N_max=282, 
                                                file_name='icdar_EXTRACTED_trocr_stage_1.csv',hugging=hugging,from_df=True, df=train_df_1)



Extracted 768 feature columns:
Extracted 768 feature columns:


In [32]:
# Modify the final classification layer (assuming you have 10 classes)

model=model_utils.get_model(selected_model, pretrained=pretrained, num_classes=num_classes,
                            input_size=768, hidden_sizes=[356])

# Define loss function and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is: ",device)
model = model.to(device)

Device is:  cuda


In [33]:
# Freeze all layers but the last n
num_trainable_layers = model_utils.get_trainable_layers(selected_model,depth=depth) 
model = training_utils.fine_tune_last_n_layers(model, num_trainable_layers)
optimizer = training_utils.get_optimizer(model, optimizer_name, lr=learning_rate)
scheduler = training_utils.get_scheduler(optimizer, scheduler_name, T_max=num_epochs)

Total Layers: 4
Total Parameters: 274,478
Trainable Parameters after freezing: 0


Unfreezing the following layers:
Linear(in_features=768, out_features=356, bias=True)
ReLU()
Linear(in_features=356, out_features=2, bias=True)


Trainable Parameters after UN-freezing last n layers: 274,478


In [34]:
# Get one batch from the dataloader
batch = next(iter(train_dataloader))

print(batch['features'][0])
print(len(batch['features'][0]))

tensor([ 5.9424e-01, -1.2593e-01,  2.8136e-01, -7.3375e-01, -6.8624e-01,
         3.5883e-01, -4.7188e-01,  3.9482e-02,  2.1203e-01,  3.6315e-01,
         5.8049e-01,  7.5142e-01, -1.3977e+00, -1.0891e+00,  3.3787e-01,
         3.3210e-01,  6.7004e-01, -1.1549e+00, -1.2098e+00, -6.6230e-01,
        -5.0766e-01,  5.9605e-02,  1.1360e+00, -7.3845e-02,  1.1692e+00,
         2.8420e-01,  1.4410e-01,  1.5071e+00,  9.5370e-01, -1.0332e-01,
         3.8128e-01,  5.7853e-01,  1.1704e+00,  3.6451e-01,  1.0471e+00,
         5.7476e-01,  9.7171e-01, -6.2705e-01,  9.5172e-01,  1.6908e-01,
        -8.2673e-01,  2.0599e-01, -2.6940e-01,  1.3058e+00,  1.0860e+00,
         6.9523e-01, -1.8471e+00, -1.7599e+00, -1.2569e-01,  8.8964e-02,
         9.1124e-01, -7.8841e-01,  9.3201e-01,  2.4149e-01,  1.5965e+00,
        -2.3184e-02,  9.5842e-01,  3.2058e-01, -5.9840e-01,  1.2297e+00,
        -2.2436e-01,  9.9887e-01,  1.4438e+00,  9.6021e-02,  6.3800e-01,
        -1.6521e+00,  3.2268e-01,  1.0563e+00, -3.2

In [35]:
start_time=datetime.now()
model,train_losses,val_losses = training_utils.train_model(model, train_dataloader, val_dataloader, criterion, optimizer, device, 
                                            num_epochs=num_epochs, 
                                            checkpoint_path=checkpoint_path,
                                            early_stopping_patience=early_stopping, scheduler=scheduler, data_type='features')
end_time=datetime.now()

Epoch 1/100: 100%|██████████| 317/317 [00:01<00:00, 166.33it/s]


Epoch 1, Train Loss: 0.6965, Train Acc: 0.5777
Epoch 1, Val Loss: 0.6708, Val Acc: 0.5707
Checkpoint saved: D:\burtm\Visual_studio_code\PD_related_projects\checkpoints\best_checkpoint.pth


Epoch 2/100: 100%|██████████| 317/317 [00:01<00:00, 181.70it/s]


Epoch 2, Train Loss: 0.6593, Train Acc: 0.6028
Epoch 2, Val Loss: 0.6690, Val Acc: 0.6000
Checkpoint saved: D:\burtm\Visual_studio_code\PD_related_projects\checkpoints\best_checkpoint.pth


Epoch 3/100: 100%|██████████| 317/317 [00:01<00:00, 194.30it/s]


...

Epoch 20, Train Loss: 0.5711, Train Acc: 0.6949
Epoch 20, Val Loss: 0.7640, Val Acc: 0.5793
Checkpoint saved: D:\burtm\Visual_studio_code\PD_related_projects\checkpoints\last_checkpoint.pth


Epoch 21/100: 100%|██████████| 317/317 [00:03<00:00, 96.38it/s] 


Epoch 21, Train Loss: 0.5687, Train Acc: 0.6891
Epoch 21, Val Loss: 0.7181, Val Acc: 0.6052
Checkpoint saved: D:\burtm\Visual_studio_code\PD_related_projects\checkpoints\last_checkpoint.pth


Epoch 22/100: 100%|██████████| 317/317 [00:01<00:00, 195.18it/s]


Epoch 22, Train Loss: 0.5674, Train Acc: 0.6937
Epoch 22, Val Loss: 0.7098, Val Acc: 0.6069
Checkpoint saved: D:\burtm\Visual_studio_code\PD_related_projects\checkpoints\last_checkpoint.pth
Early stopping triggered.


In [ ]:
#get the best_checkpoint.pth and add the training metadata to it
checkpoint = torch.load(checkpoint_path+'best_checkpoint.pth')
checkpoint['training_metadata'] = training_metadata
val_accuracy= checkpoint['val_acc']
# Save the updated checkpoint with metadata
torch.save(checkpoint, checkpoint_path)

#do the same for the last checkpoint
checkpoint = torch.load(checkpoint_path+'last_checkpoint.pth')
checkpoint['training_metadata'] = training_metadata
# Save the updated checkpoint with metadata
torch.save(checkpoint, checkpoint_path)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
torch.save(checkpoint, f"{models_path}\{training_metadata['type_of_approach_sigla']}_ValAcc{val_accuracy}_{timestamp}.pth")

# easy access

In [23]:
def reload_modules():
    import importlib
    import utils.data_loading as data_loading
    import utils.visualization as visualization
    import utils.dataframes as dataframes
    import utils.utils_transforms as u_transforms
    import utils.training_utils as training_utils
    import utils.model_utils as model_utils
    

    importlib.reload(data_loading)
    importlib.reload(visualization)
    importlib.reload(dataframes)
    importlib.reload(u_transforms)
    importlib.reload(model_utils)
    importlib.reload(training_utils)

    return data_loading, visualization, dataframes, u_transforms, training_utils, model_utils
data_loading, visualization, dataframes, u_transforms, training_utils, model_utils = reload_modules()